In [759]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [760]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [761]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [762]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [763]:
STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  

STARTINGCSV=pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedPoints,shorthandedGoals,opponentAbbrev,pim,toi,playerId,firstName,lastName
10123,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,0,0,0,0,0,0,17,0,0,NSH,0,13:41,8475177,Calvin,de Haan
4160,2023020003,VGK,H,2023-10-10,1,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,1,0,0,0,0,3,23,0,0,SEA,0,19:53,8478403,Jack,Eichel
10676,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,-1,0,0,0,0,1,18,0,0,NSH,2,11:42,8480246,Nick,Perbix
8731,2023020001,NSH,R,2023-10-10,1,0,{'default': 'Predators'},{'default': 'Lightning'},1,1,0,0,0,0,2,20,0,0,TBL,0,13:43,8481704,Juuso,Parssinen
10216,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,0,0,0,0,0,1,23,0,0,NSH,0,14:01,8476822,Luke,Glendening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,2023020697,CGY,H,2024-01-18,0,0,{'default': 'Flames'},{'default': 'Maple Leafs'},0,0,0,0,0,0,0,8,0,0,TOR,0,5:35,8479346,Dillon,Dube
863,2023020701,NYR,R,2024-01-18,0,0,{'default': 'Rangers'},{'default': 'Golden Knights'},0,-1,0,0,0,0,2,23,0,0,VGK,4,24:15,8479323,Adam,Fox
1506,2023020697,TOR,R,2024-01-18,0,0,{'default': 'Maple Leafs'},{'default': 'Flames'},0,-2,0,0,0,0,5,23,0,0,CGY,0,18:19,8475166,John,Tavares
1497,2023020697,TOR,R,2024-01-18,0,1,{'default': 'Maple Leafs'},{'default': 'Flames'},1,-1,0,0,0,0,1,34,0,0,CGY,0,24:35,8474673,TJ,Brodie


In [764]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,Vancouver,Canucks,VAN
1,Boston,Bruins,BOS
2,Winnipeg,Jets,WPG
3,Colorado,Avalanche,COL
4,Florida,Panthers,FLA
5,NY Rangers,Rangers,NYR
6,Dallas,Stars,DAL
7,Vegas,Golden Knights,VGK
8,Philadelphia,Flyers,PHI
9,Carolina,Hurricanes,CAR


In [765]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','homeRoadFlag','shots','opponentAbbrev','toi']]
playerStats['toi']=pd.to_numeric(playerStats['toi'].str.slice(0,2).str.replace(':', '', regex=False)
                                 )+(pd.to_numeric(playerStats['toi'].str.slice(3, 5))/60) #min+sec in decimal base not base60

playerStats['homeRoadFlag'] = np.where(playerStats['homeRoadFlag'] =='H', 1, 0)

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)
playerStats['HomeGames'] = playerStats.groupby(['playerId']).homeRoadFlag.transform(np.sum)
playerStats['AwayGames']=playerStats['GP']-playerStats['HomeGames']
playerStats['TotalShotsSzn']=playerStats.groupby(['playerId']).shots.transform(np.sum)
playerStats['SeasonAvgSOG']= playerStats['TotalShotsSzn']/playerStats['GP']
#playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['AvgShotsSznByLoc']=playerStats.groupby(['playerId','homeRoadFlag']).shots.transform(np.mean)

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)
playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)
playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).shots.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)

playerStats['AvgShotsAgainstThisOpp']=playerStats.groupby(['playerId','opponentAbbrev']).shots.transform(np.mean)

homeawaystats=playerStats[['playerId','opponentAbbrev','gameDate','homeRoadFlag','shots']]
homeawaystats['totalshotsvsOppAtLoc']=homeawaystats.groupby(['playerId','opponentAbbrev','homeRoadFlag']).shots.transform(np.sum)
homeawaystats['avgshotsvsOppAtLoc']=homeawaystats.groupby(['playerId','opponentAbbrev','homeRoadFlag']).shots.transform(np.mean)

homeawaystats['GPHomeAwayvsOpp'] = homeawaystats.groupby(['playerId','opponentAbbrev','homeRoadFlag'])['gameDate'].cumcount(ascending=False)+1
homeawaystats['GPHomeAwayvsOpp'] = homeawaystats.groupby(['playerId','opponentAbbrev','homeRoadFlag']).GPHomeAwayvsOpp.transform(np.max)

playerStats=playerStats.merge(homeawaystats[['playerId','opponentAbbrev','homeRoadFlag','GPHomeAwayvsOpp','avgshotsvsOppAtLoc','totalshotsvsOppAtLoc']],on=['playerId','opponentAbbrev','homeRoadFlag'],how='left')
playerStats['avgshotsvsOppAtLoc']=np.where(playerStats['avgshotsvsOppAtLoc'].isna(), 0, playerStats['avgshotsvsOppAtLoc'])
playerStats['totalshotsvsOppAtLoc']=np.where(playerStats['totalshotsvsOppAtLoc'].isna(), 0, playerStats['totalshotsvsOppAtLoc'])


playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,homeRoadFlag,shots,opponentAbbrev,toi,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,AvgShotsAgainstThisOpp,GPHomeAwayvsOpp,avgshotsvsOppAtLoc,totalshotsvsOppAtLoc
0,WPG,8475455,Brenden,Dillon,2023-12-31,0,1,MIN,19.533333,43,43,24,19,40,0.930233,1.052632,1,0.6,1,0.9,1.000000,1,1.0,1
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,1,MIN,14.283333,42,43,24,19,40,0.930233,0.833333,1,0.6,1,0.9,1.000000,1,1.0,1
2,WPG,8475455,Brenden,Dillon,2023-12-27,0,0,CHI,17.500000,41,43,24,19,40,0.930233,1.052632,1,0.6,1,0.9,0.333333,1,0.0,0
3,WPG,8475455,Brenden,Dillon,2023-12-22,1,0,BOS,18.350000,40,43,24,19,40,0.930233,0.833333,1,0.6,1,0.9,0.000000,1,0.0,0
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,1,DET,20.166667,39,43,24,19,40,0.930233,0.833333,1,0.6,1,0.9,0.500000,1,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29172,SJS,8482667,William,Eklund,2024-01-09,0,2,TOR,15.016667,4,45,20,25,72,1.600000,1.708333,0,2.6,0,2.3,1.000000,1,2.0,2
29173,SJS,8482667,William,Eklund,2024-01-06,1,0,TOR,17.966667,3,45,20,25,72,1.600000,1.550000,0,2.6,0,2.3,1.000000,1,0.0,0
29174,SJS,8482667,William,Eklund,2024-01-04,1,0,WPG,19.916667,2,45,20,25,72,1.600000,1.550000,0,2.6,0,2.3,1.500000,2,1.5,3
29175,SJS,8482667,William,Eklund,2024-01-04,1,0,WPG,19.916667,2,45,20,25,72,1.600000,1.550000,0,2.6,0,2.3,1.500000,2,1.5,3


In [766]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

#toi
res = playerStats.groupby(['playerId'])["toi"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=ShotsNoOutliers.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['toi'])&(res['toi']<res['High'])].drop(columns=['Low', 'High'])


ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,homeRoadFlag,shots,opponentAbbrev,toi,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,AvgShotsAgainstThisOpp,GPHomeAwayvsOpp,avgshotsvsOppAtLoc,totalshotsvsOppAtLoc,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,WPG,8475455,Brenden,Dillon,2023-12-31,0,1,MIN,19.533333,43,43,24,19,40,0.930233,1.052632,1,0.6,1,0.9,1.0,1,1.0,1,22,22,1.272727
2,WPG,8475455,Brenden,Dillon,2023-12-20,1,1,DET,20.166667,39,43,24,19,40,0.930233,0.833333,1,0.6,1,0.9,0.5,1,1.0,1,21,22,1.272727
4,WPG,8475455,Brenden,Dillon,2023-12-13,0,2,LAK,16.933333,36,43,24,19,40,0.930233,1.052632,0,0.6,1,0.9,1.5,1,2.0,2,20,22,1.272727
7,WPG,8475455,Brenden,Dillon,2023-12-10,0,2,ANA,17.116667,34,43,24,19,40,0.930233,1.052632,0,0.6,1,0.9,1.5,2,1.5,3,19,22,1.272727
8,WPG,8475455,Brenden,Dillon,2023-12-10,0,2,ANA,17.116667,34,43,24,19,40,0.930233,1.052632,0,0.6,1,0.9,1.5,2,1.5,3,18,22,1.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18616,SJS,8482667,William,Eklund,2024-01-15,0,3,BUF,18.600000,7,45,20,25,72,1.600000,1.708333,0,2.6,0,2.3,3.0,1,3.0,3,5,34,1.764706
18617,SJS,8482667,William,Eklund,2024-01-13,0,2,OTT,19.916667,6,45,20,25,72,1.600000,1.708333,0,2.6,0,2.3,2.0,1,2.0,2,4,34,1.764706
18618,SJS,8482667,William,Eklund,2024-01-11,0,3,MTL,18.216667,5,45,20,25,72,1.600000,1.708333,0,2.6,0,2.3,2.5,1,3.0,3,3,34,1.764706
18619,SJS,8482667,William,Eklund,2024-01-09,0,2,TOR,15.016667,4,45,20,25,72,1.600000,1.708333,0,2.6,0,2.3,1.0,1,2.0,2,2,34,1.764706


In [767]:
Teams

,City,TeamName,TeamID
0,Vancouver,Canucks,VAN
1,Boston,Bruins,BOS
2,Winnipeg,Jets,WPG
3,Colorado,Avalanche,COL
4,Florida,Panthers,FLA
5,NY Rangers,Rangers,NYR
6,Dallas,Stars,DAL
7,Vegas,Golden Knights,VGK
8,Philadelphia,Flyers,PHI
9,Carolina,Hurricanes,CAR


In [768]:
import datetime
# todaysindex=datetime.datetime.today().weekday()+2
# if  todaysindex>6:
#     todaysindex-=7
#I think todaysindex is always 3
todaysindex=3

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,DET,CAR
1,NJD,CBJ
2,MIN,FLA
3,NYI,CHI


In [769]:
shotSummary=ShotsNoOutliers[['teamAbbrev', 'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots','LastTenAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['teamAbbrev'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='teamAbbrev', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])
todaysSkaters['HomeFlag'] = np.where(todaysSkaters['teamAbbrev'].isin(TodaysGames['HomeTeam'].tolist()), 1, 0)

jointhis=playerStats[['playerId','opponentAbbrev','homeRoadFlag','GPHomeAwayvsOpp','avgshotsvsOppAtLoc','totalshotsvsOppAtLoc']].rename(
    columns={"opponentAbbrev": "Opponent","homeRoadFlag": "HomeFlag"})

todaysSkaters=todaysSkaters.merge(jointhis,on=['playerId','Opponent','HomeFlag'],how='left')
todaysSkaters=todaysSkaters.merge(playerStats[['playerId','homeRoadFlag','AvgShotsSznByLoc']].rename(
    columns={"homeRoadFlag": "HomeFlag"}),on=['playerId','HomeFlag'],how='left').drop_duplicates()
todaysSkaters=todaysSkaters.merge(playerStats[['playerId','opponentAbbrev','AvgShotsAgainstThisOpp']].rename(
    columns={"opponentAbbrev": "Opponent"}),on=['playerId','Opponent'],how='left').drop_duplicates()

todaysSkaters['avgshotsvsOppAtLoc']=np.where(todaysSkaters['avgshotsvsOppAtLoc'].isna(), 0, todaysSkaters['avgshotsvsOppAtLoc'])
todaysSkaters['AvgShotsAgainstThisOpp']=np.where(todaysSkaters['AvgShotsAgainstThisOpp'].isna(), 0, todaysSkaters['AvgShotsAgainstThisOpp'])
todaysSkaters['GPHomeAwayvsOpp']=np.where(todaysSkaters['GPHomeAwayvsOpp'].isna(), 0, todaysSkaters['GPHomeAwayvsOpp'])

todaysSkaters=todaysSkaters[['teamAbbrev',	'playerId',	'firstName',	'lastName',	'SeasonAvgSOG','AvgShotsSznByLoc',
                             	'LastFiveAvgShots',	'LastTenAvgShots',	'NoOutliersAvgSOG',	'Opponent',
                                    	'HomeFlag',	'GPHomeAwayvsOpp',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp']]

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,Opponent,HomeFlag,GPHomeAwayvsOpp,avgshotsvsOppAtLoc,AvgShotsAgainstThisOpp
0,NJD,8481559,Jack,Hughes,3.285714,3.928571,0.0,0.4,4.607143,CBJ,0,1.0,3.0,6.333333
1,DET,8477946,Dylan,Larkin,3.022727,3.666667,0.0,1.7,3.612903,CAR,0,0.0,0.0,0.000000
2,MIN,8478493,Joel,Eriksson Ek,3.840909,3.380952,5.2,4.7,3.600000,FLA,0,0.0,0.0,2.000000
3,DET,8474141,Patrick,Kane,1.431818,2.700000,0.0,0.0,3.437500,CAR,0,0.0,0.0,2.000000
4,FLA,8479314,Matthew,Tkachuk,3.704545,3.761905,3.6,2.7,3.343750,MIN,1,0.0,0.0,10.000000
5,FLA,8477409,Carter,Verhaeghe,3.295455,3.571429,3.6,3.8,3.307692,MIN,1,0.0,0.0,3.000000
6,NYI,8478445,Mathew,Barzal,2.931818,2.428571,3.0,2.6,3.225806,CHI,0,0.0,0.0,0.000000
7,NYI,8475754,Brock,Nelson,3.159091,3.318182,4.2,3.9,3.212121,CHI,0,0.0,0.0,0.000000
8,CBJ,8476432,Boone,Jenner,1.840909,2.125000,0.0,0.0,3.095238,NJD,1,0.0,0.0,2.000000
9,MIN,8478864,Kirill,Kaprizov,2.795455,3.473684,0.0,1.1,3.041667,FLA,0,0.0,0.0,4.000000


In [770]:
goalieSTARTINGCSV2023=pd.read_csv('GoalieLastYear2023Stats.csv')  
goalieSTARTINGCSV2024=pd.read_csv('GoalieThisYear2024Stats.csv')  

goalieSTARTINGCSV=pd.concat([goalieSTARTINGCSV2023, goalieSTARTINGCSV2024])
goalieSTARTINGCSV=goalieSTARTINGCSV[goalieSTARTINGCSV['gamesStarted']==1]
goalieSTARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,gamesStarted,decision,shotsAgainst,goalsAgainst,savePctg,shutouts,opponentAbbrev,pim,toi,playerId,firstName,lastName
1098,2023020002,CHI,R,2023-10-10,0,0,{'default': 'Blackhawks'},{'default': 'Penguins'},1,W,41,2,0.951220,0,PIT,0,60:00,8475852,Petr,Mrazek
270,2023020003,VGK,H,2023-10-10,0,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,W,33,1,0.969697,0,SEA,0,60:00,8478499,Adin,Hill
760,2023020003,SEA,R,2023-10-10,0,0,{'default': 'Kraken'},{'default': 'Golden Knights'},1,L,27,3,0.888889,0,VGK,0,59:37,8475831,Philipp,Grubauer
586,2023020002,PIT,H,2023-10-10,0,0,{'default': 'Penguins'},{'default': 'Blackhawks'},1,L,35,3,0.914286,0,CHI,0,57:30,8477465,Tristan,Jarry
672,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},1,W,31,3,0.903226,0,NSH,0,60:00,8477992,Jonas,Johansson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2023020700,ARI,R,2024-01-18,0,0,{'default': 'Coyotes'},{'default': 'Canucks'},1,L,26,2,0.923077,0,VAN,0,58:51,8478971,Connor,Ingram
61,2023020701,VGK,H,2024-01-18,0,0,{'default': 'Golden Knights'},{'default': 'Rangers'},1,W,30,1,0.966667,0,NYR,0,59:55,8480313,Logan,Thompson
225,2023020693,OTT,H,2024-01-18,0,0,"{'default': 'Senators', 'fr': 'Sénateurs'}",{'default': 'Canadiens'},1,W,23,2,0.913043,0,MTL,0,59:55,8476914,Joonas,Korpisalo
106,2023020699,NSH,R,2024-01-18,0,0,{'default': 'Predators'},{'default': 'Kings'},1,W,34,1,0.970588,0,LAK,0,59:52,8477424,Juuse,Saros


In [771]:
goalieStats=goalieSTARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','opponentAbbrev','gamesStarted','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['PlayerGameNumber'] = goalieStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
goalieStats['TeamGameNumber'] = goalieStats.groupby(['teamAbbrev'])['gameDate'].cumcount(ascending=False)+1
goalieStats['PlayerGP']=goalieStats.groupby(['playerId']).gamesStarted.transform(np.sum)
goalieStats['TeamGP']=goalieStats.groupby(['teamAbbrev']).gamesStarted.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['teamAbbrev']).shotsAgainst.transform(np.sum)/goalieStats['TeamGP']
goalieStats['RankAvgShotsAgainst']=goalieStats['AvgShotsAgainst'].rank(pct=True)*100

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgGoalsAgainst']=goalieStats['AvgGoalsAgainst'].rank(pct=True)*100

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgSavePctg']=goalieStats['AvgSavePctg'].rank(pct=True)*100

goalieStats['LastFiveFlag'] = np.where(goalieStats['TeamGameNumber'] >= goalieStats['TeamGP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['teamAbbrev','PlayerGP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['teamAbbrev'])['PlayerGP'].rank().astype(int)
summaryGoalieStats=summaryGoalieStats[['teamAbbrev','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','teamAbbrev':'GoalieteamAbbrev'})
summaryGoalieStats

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,WPG,1,Laurent,Brossoit,38.507463,4.253731,83.731343
9,WPG,2,Connor,Hellebuyck,38.507463,5.858209,91.305970
35,BOS,1,Linus,Ullmark,75.037313,45.559701,89.365672
53,BOS,2,Jeremy,Swayman,75.037313,12.276119,96.455224
71,VAN,2,Thatcher,Demko,66.865672,22.425373,86.455224
...,...,...,...,...,...,...,...
1115,SJS,1,Kaapo,Kahkonen,95.111940,88.917910,25.111940
1133,SJS,2,Mackenzie,Blackwood,95.111940,87.276119,28.022388
33,COL,1,Justus,Annunen,19.440299,99.179104,58.582090
91,TOR,1,Ilya,Samsonov,82.574627,68.395522,9.402985


In [772]:
full=todaysSkaters.merge(summaryGoalieStats
                         [summaryGoalieStats['GPRank']==1]
                         , left_on='Opponent', right_on='GoalieteamAbbrev', how='left').drop(columns=['GoalieteamAbbrev','GPRank'])
full=full[['teamAbbrev', 'HomeFlag', 'firstName', 'lastName', 'NoOutliersAvgSOG','AvgShotsSznByLoc',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
        'GoalieLastName',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp',
        'SeasonAvgSOG', 'playerId', 	
       'RankAvgGoalsAgainst', 'RankAvgSavePctg']].rename(columns={"NoOutliersAvgSOG": "ExpectedSOG"}).round(2)

full.sort_values(by=['ExpectedSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(25)

,teamAbbrev,HomeFlag,firstName,lastName,ExpectedSOG,AvgShotsSznByLoc,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,avgshotsvsOppAtLoc,AvgShotsAgainstThisOpp,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg
0,NJD,0,Jack,Hughes,4.61,3.93,0.0,0.4,CBJ,88.54,Tarasov,3.0,6.33,3.29,8481559,99.66,7.35
1,DET,0,Dylan,Larkin,3.61,3.67,0.0,1.7,CAR,1.64,Andersen,0.0,0.00,3.02,8477946,30.78,0.93
2,MIN,0,Joel,Eriksson Ek,3.60,3.38,5.2,4.7,FLA,7.57,Stolarz,0.0,2.00,3.84,8478493,8.88,66.72
3,DET,0,Patrick,Kane,3.44,2.70,0.0,0.0,CAR,1.64,Andersen,0.0,2.00,1.43,8474141,30.78,0.93
4,FLA,1,Matthew,Tkachuk,3.34,3.76,3.6,2.7,MIN,44.78,Fleury,0.0,10.00,3.70,8479314,78.88,36.19
5,FLA,1,Carter,Verhaeghe,3.31,3.57,3.6,3.8,MIN,44.78,Fleury,0.0,3.00,3.30,8477409,78.88,36.19
6,NYI,0,Mathew,Barzal,3.23,2.43,3.0,2.6,CHI,85.30,Soderblom,0.0,0.00,2.93,8478445,98.36,10.07
7,NYI,0,Brock,Nelson,3.21,3.32,4.2,3.9,CHI,85.30,Soderblom,0.0,0.00,3.16,8475754,98.36,10.07
8,CBJ,1,Boone,Jenner,3.10,2.12,0.0,0.0,NJD,13.43,Daws,0.0,2.00,1.84,8476432,40.93,83.10
9,MIN,0,Kirill,Kaprizov,3.04,3.47,0.0,1.1,FLA,7.57,Stolarz,0.0,4.00,2.80,8478864,8.88,66.72


In [773]:
margin=full
margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['ExpectedSOG']

margin=margin[['teamAbbrev', 'HomeFlag', 'firstName', 'lastName', 'ExpectedSOG','AvgShotsSznByLoc',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
        'GoalieLastName',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp',
        'SeasonAvgSOG', 'playerId', 
       'RankAvgGoalsAgainst', 'RankAvgSavePctg', 'lastfivemargin']]

margin.sort_values(by=['lastfivemargin']
                        , ascending=False #or True for players trending down
                        #, ascending=True
                       ).reset_index(drop=True).head(25)

,teamAbbrev,HomeFlag,firstName,lastName,ExpectedSOG,AvgShotsSznByLoc,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,avgshotsvsOppAtLoc,AvgShotsAgainstThisOpp,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg,lastfivemargin
0,MIN,0,Joel,Eriksson Ek,3.60,3.38,5.2,4.7,FLA,7.57,Stolarz,0.0,2.00,3.84,8478493,8.88,66.72,1.60
1,MIN,0,Marcus,Johansson,1.88,1.86,3.2,2.3,FLA,7.57,Stolarz,0.0,0.00,1.70,8475149,8.88,66.72,1.32
2,CAR,1,Brady,Skjei,1.89,2.30,3.2,3.0,DET,79.93,Reimer,0.0,3.00,2.09,8476869,92.24,47.16,1.31
3,FLA,1,Ryan,Lomberg,1.22,1.33,2.4,1.3,MIN,44.78,Fleury,0.0,2.00,1.11,8479066,78.88,36.19,1.18
4,CAR,1,Brent,Burns,2.37,2.40,3.4,2.5,DET,79.93,Reimer,0.0,4.00,2.56,8470613,92.24,47.16,1.03
5,NYI,0,Brock,Nelson,3.21,3.32,4.2,3.9,CHI,85.30,Soderblom,0.0,0.00,3.16,8475754,98.36,10.07,0.99
6,CBJ,1,Justin,Danforth,1.42,1.12,2.4,1.9,NJD,13.43,Daws,3.0,2.00,1.30,8479941,40.93,83.10,0.98
7,NYI,0,Jean-Gabriel,Pageau,1.46,1.18,2.4,2.0,CHI,85.30,Soderblom,0.0,0.00,1.20,8476419,98.36,10.07,0.94
8,FLA,1,Evan,Rodrigues,2.07,2.14,3.0,2.3,MIN,44.78,Fleury,0.0,4.00,2.23,8478542,78.88,36.19,0.93
9,DET,0,Alex,DeBrincat,2.72,2.55,3.6,3.7,CAR,1.64,Andersen,0.0,4.00,2.93,8479337,30.78,0.93,0.88


In [774]:
margin['locationmargin']=margin['AvgShotsSznByLoc']-margin['ExpectedSOG']

margin.sort_values(by=['locationmargin']
                        , ascending=False #or True for players trending down
                        #, ascending=True
                       ).reset_index(drop=True).head(25)

,teamAbbrev,HomeFlag,firstName,lastName,ExpectedSOG,AvgShotsSznByLoc,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,avgshotsvsOppAtLoc,AvgShotsAgainstThisOpp,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg,lastfivemargin,locationmargin
0,NJD,0,Dougie,Hamilton,2.77,3.89,0.0,0.0,CBJ,88.54,Tarasov,0.0,3.00,1.45,8476462,99.66,7.35,-2.77,1.12
1,CAR,1,Martin,Necas,2.35,2.94,0.0,1.2,DET,79.93,Reimer,0.0,3.00,2.07,8480039,92.24,47.16,-2.35,0.59
2,CAR,1,Tony,DeAngelo,1.33,1.88,0.0,0.0,DET,79.93,Reimer,0.0,0.00,0.77,8477950,92.24,47.16,-1.33,0.55
3,CHI,1,Colin,Blackwell,1.89,2.43,0.0,0.0,NYI,98.40,Varlamov,0.0,0.00,0.73,8476278,36.98,88.13,-1.89,0.54
4,FLA,1,Brandon,Montour,2.17,2.71,0.0,0.0,MIN,44.78,Fleury,0.0,0.00,1.57,8477986,78.88,36.19,-2.17,0.54
5,CBJ,1,Adam,Fantilli,2.00,2.52,2.4,2.3,NJD,13.43,Daws,2.0,1.33,2.41,8484166,40.93,83.10,0.40,0.52
6,CAR,1,Sebastian,Aho,2.63,3.15,1.6,2.3,DET,79.93,Reimer,0.0,3.00,2.81,8478427,92.24,47.16,-1.03,0.52
7,NJD,0,Tomas,Nosek,1.00,1.50,0.0,0.0,CBJ,88.54,Tarasov,0.0,0.00,0.10,8477931,99.66,7.35,-1.00,0.50
8,MIN,0,Kirill,Kaprizov,3.04,3.47,0.0,1.1,FLA,7.57,Stolarz,0.0,4.00,2.80,8478864,8.88,66.72,-3.04,0.43
9,FLA,1,Matthew,Tkachuk,3.34,3.76,3.6,2.7,MIN,44.78,Fleury,0.0,10.00,3.70,8479314,78.88,36.19,0.26,0.42


In [775]:
margin['combinedmargin']=margin['lastfivemargin']+margin['locationmargin']

margin.sort_values(by=['combinedmargin']
                        , ascending=False #or True for players trending down
                        #, ascending=True
                       ).reset_index(drop=True).head(25)

,teamAbbrev,HomeFlag,firstName,lastName,ExpectedSOG,AvgShotsSznByLoc,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,avgshotsvsOppAtLoc,AvgShotsAgainstThisOpp,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg,lastfivemargin,locationmargin,combinedmargin
0,CAR,1,Brady,Skjei,1.89,2.30,3.2,3.0,DET,79.93,Reimer,0.0,3.00,2.09,8476869,92.24,47.16,1.31,0.41,1.72
1,MIN,0,Joel,Eriksson Ek,3.60,3.38,5.2,4.7,FLA,7.57,Stolarz,0.0,2.00,3.84,8478493,8.88,66.72,1.60,-0.22,1.38
2,MIN,0,Marcus,Johansson,1.88,1.86,3.2,2.3,FLA,7.57,Stolarz,0.0,0.00,1.70,8475149,8.88,66.72,1.32,-0.02,1.30
3,FLA,1,Ryan,Lomberg,1.22,1.33,2.4,1.3,MIN,44.78,Fleury,0.0,2.00,1.11,8479066,78.88,36.19,1.18,0.11,1.29
4,NYI,0,Brock,Nelson,3.21,3.32,4.2,3.9,CHI,85.30,Soderblom,0.0,0.00,3.16,8475754,98.36,10.07,0.99,0.11,1.10
5,CAR,1,Brent,Burns,2.37,2.40,3.4,2.5,DET,79.93,Reimer,0.0,4.00,2.56,8470613,92.24,47.16,1.03,0.03,1.06
6,MIN,0,Marco,Rossi,1.93,2.10,2.8,2.4,FLA,7.57,Stolarz,0.0,2.00,2.00,8482079,8.88,66.72,0.87,0.17,1.04
7,FLA,1,Evan,Rodrigues,2.07,2.14,3.0,2.3,MIN,44.78,Fleury,0.0,4.00,2.23,8478542,78.88,36.19,0.93,0.07,1.00
8,CBJ,1,Adam,Fantilli,2.00,2.52,2.4,2.3,NJD,13.43,Daws,2.0,1.33,2.41,8484166,40.93,83.10,0.40,0.52,0.92
9,NYI,0,Anders,Lee,2.15,2.14,3.0,3.3,CHI,85.30,Soderblom,0.0,0.00,2.43,8475314,98.36,10.07,0.85,-0.01,0.84


In [776]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df